In [191]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

ddf_uground_shared = dd.read_parquet("output/uground_shared")
ddf_uground_multi = dd.read_parquet("output/uground_multi")


# 국토부 제공 데이터 및 해설

## 실태조사 대상 선정 기준

### 단독(다가구, 다중)

아래 조건 중 하나 이상 해당하는 경우 선정

- 단독주택(다가구, 다중) 중 지하층 용도가 다가구주택, 다중주택인 건물동
    - 층 용도: 건축물대장 층별개요의 주_용도_코드
        - 01002: 다중주택
        - 01003: 다가구주택
    - 제공 파일에서는 (이 경우에 해당하는 경우) 층_용도로 표출
    - 건물동 용도는 대분류(단독주택 01000)로 되어있기 때문에 소분류로 기재된 층 용도 사용
- 지하_층_수가 1 이상이면서 세대_수 또는 가구_수가 2 이상인 단독주택(대분류) 건물동
    - 제공 파일에서는 (이 경우에 해당하는 경우) 표제부_세대_수, 표제부_가구_수로 표출
- 지하_층_수가 1 이상이면서 기타_용도 텍스트에 "다가구", "다중"이 포함된 건물동
    - 제공 파일에서는 (이 경우에 해당하는 경우) 표제부_기타_용도로 표출

### 공동(다세대, 연립)

아래 조건 중 하나 이상 해당하는 경우 선정

- 집합건축물 중 지하층 전유부의 용도가 주택인 건물동
    - 전유부 용도: 건축물대장 전유공유면적의 주_용도_코드
        - 02002: 연립주택
        - 02003: 다세대주택
    - 제공 파일에서는 (이 경우에 해당하는 경우) 전유부_용도, 지하_주택_전유부_수로 표출
- 공동주택(다세대, 연립) 중 지하층 용도가 다세대주택, 연립주택인 건물동
    - 제공 파일에서는 (이 경우에 해당하는 경우) 층_용도로 표출
- 지하_층_수가 1 이상이면서 기타_용도 텍스트에 "다세대", "연립"이 포함된 건물동
    - 제공 파일에서는 (이 경우에 해당하는 경우) 표제부_기타_용도로 표출


## 시도, 시군구별 집계

In [192]:
df_sido_name = pd.read_csv("output/code_sido.csv", dtype=str)
df_sido_name = df_sido_name[df_sido_name.폐지여부 == "존재"]
s_sido = df_sido_name.set_index("시도코드").시도명
s_sido["36"] = "세종특별자치시"
s_sido


시도코드
11      서울특별시
26      부산광역시
27      대구광역시
28      인천광역시
29      광주광역시
30      대전광역시
31      울산광역시
41        경기도
42        강원도
43       충청북도
44       충청남도
45       전라북도
46       전라남도
47       경상북도
48       경상남도
50    제주특별자치도
36    세종특별자치시
Name: 시도명, dtype: object

In [193]:
df_sgg_name = pd.read_csv("output/code_sgg.csv", dtype=str)
df_sgg_name = df_sgg_name.set_index("시군구코드")
s_sgg = df_sgg_name.시군구명
s_sgg


시군구코드
11000           서울특별시
11110       서울특별시 종로구
11140        서울특별시 중구
11170       서울특별시 용산구
11200       서울특별시 성동구
             ...     
49710        제주도 북제주군
49720        제주도 남제주군
50000         제주특별자치도
50110     제주특별자치도 제주시
50130    제주특별자치도 서귀포시
Name: 시군구명, Length: 461, dtype: object

In [194]:
ddf_uground_shared.columns


Index(['층_용도', '최저_지하_주택_층', '지하_주택_층별개요_수', '표제부_세대_수', '표제부_가구_수',
       '표제부_주_용도', '표제부_기타_용도', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드',
       '대장_종류_코드_명', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드',
       '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '외필지_수', '새주소_도로_코드',
       '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '동_명', '주_부속_구분_코드',
       '주_부속_구분_코드_명', '대지_면적(㎡)', '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)',
       '용적_률_산정_연면적(㎡)', '용적_률(%)', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드',
       '주_용도_코드_명', '기타_용도', '지붕_코드', '지붕_코드_명', '기타_지붕', '세대_수(세대)',
       '가구_수(가구)', '높이(m)', '지상_층_수', '지하_층_수', '승용_승강기_수', '비상용_승강기_수',
       '부속_건축물_수', '부속_건축물_면적(㎡)', '총_동_연면적(㎡)', '옥내_기계식_대수(대)',
       '옥내_기계식_면적(㎡)', '옥외_기계식_대수(대)', '옥외_기계식_면적(㎡)', '옥내_자주식_대수(대)',
       '옥내_자주식_면적(㎡)', '옥외_자주식_대수(대)', '옥외_자주식_면적(㎡)', '허가_일', '착공_일',
       '사용승인_일', '허가번호_년', '허가번호_기관_코드', '허가번호_기관_코드_명', '허가번호_구분_코드',
       '허가번호_구분_코드_명', '호_수(호)', '에너지효율_등급', '에너지절감_율', '에너지_EPI점수',
     

In [195]:
ddf_sgg_shared = ddf_uground_shared.시군구_코드.to_frame()
ddf_sgg_shared


,시군구_코드
npartitions=46,
,object
,...
...,...
,...
,...


In [196]:
ddf_sgg_shared.head()


,시군구_코드
관리_건축물대장_PK,
11110-100179949,11110
11110-100192807,11110
11110-100209165,11110
11110-10710,11110
11110-11031,11110


In [197]:
ddf_sgg_shared["시도_코드"] = ddf_sgg_shared.시군구_코드.str[:2]
ddf_sgg_shared.head()


,시군구_코드,시도_코드
관리_건축물대장_PK,,
11110-100179949,11110,11
11110-100192807,11110,11
11110-100209165,11110,11
11110-10710,11110,11
11110-11031,11110,11


In [198]:
ddf_sgg_shared["cnt"] = 1
df_sgg_shared = ddf_sgg_shared.groupby(["시도_코드", "시군구_코드"]).count().compute()
df_sgg_shared["cat"] = "다가구_다중"
df_sgg_shared = df_sgg_shared.sort_values("시군구_코드")
df_sgg_shared


cnt     cat
시도_코드 시군구_코드              
11    11110   1939  다가구_다중
      11140   1172  다가구_다중
      11170   3259  다가구_다중
      11200   2952  다가구_다중
      11215   5397  다가구_다중
...            ...     ...
48    48870     11  다가구_다중
      48880      7  다가구_다중
      48890      1  다가구_다중
50    50110   1523  다가구_다중
      50130    402  다가구_다중

[245 rows x 2 columns]

In [199]:
df_sido_shared = df_sgg_shared.groupby("시도_코드").agg({"cnt": "sum", "cat": "first"})
df_sido_shared["시군구_코드"] = df_sido_shared.index.str[:] + "000"
df_sido_shared


,cnt,cat,시군구_코드
시도_코드,,,
11,107344,다가구_다중,11000
26,5337,다가구_다중,26000
27,7072,다가구_다중,27000
28,11678,다가구_다중,28000
29,5462,다가구_다중,29000
30,12212,다가구_다중,30000
31,842,다가구_다중,31000
36,100,다가구_다중,36000
41,68727,다가구_다중,41000


In [200]:
ddf_uground_multi.columns


Index(['전유부_용도', '최저_지하_주택_전유부_층', '지하_주택_전유부_수', '층_용도', '최저_지하_주택_층',
       '지하_주택_층별개요_수', '표제부_세대_수', '표제부_가구_수', '표제부_주_용도', '표제부_기타_용도',
       '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명', '대지_위치',
       '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명',
       '블록', '로트', '외필지_수', '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드',
       '새주소_본_번', '새주소_부_번', '동_명', '주_부속_구분_코드', '주_부속_구분_코드_명', '대지_면적(㎡)',
       '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)', '용적_률_산정_연면적(㎡)', '용적_률(%)', '구조_코드',
       '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명', '기타_용도', '지붕_코드', '지붕_코드_명',
       '기타_지붕', '세대_수(세대)', '가구_수(가구)', '높이(m)', '지상_층_수', '지하_층_수',
       '승용_승강기_수', '비상용_승강기_수', '부속_건축물_수', '부속_건축물_면적(㎡)', '총_동_연면적(㎡)',
       '옥내_기계식_대수(대)', '옥내_기계식_면적(㎡)', '옥외_기계식_대수(대)', '옥외_기계식_면적(㎡)',
       '옥내_자주식_대수(대)', '옥내_자주식_면적(㎡)', '옥외_자주식_대수(대)', '옥외_자주식_면적(㎡)', '허가_일',
       '착공_일', '사용승인_일', '허가번호_년', '허가번호_기관_코드', '허가번호_기관_코드_명', '허가번호_구분_코드',
       '허가번호_구분_코드_명', '호_수(호)'

In [201]:
ddf_sgg_multi = ddf_uground_multi[["시군구_코드", "지하_주택_전유부_수"]]
ddf_sgg_multi = ddf_sgg_multi.astype({"지하_주택_전유부_수": "Int64"})
ddf_sgg_multi["시도_코드"] = ddf_sgg_multi.시군구_코드.str[:2]
ddf_sgg_multi


,시군구_코드,지하_주택_전유부_수,시도_코드
npartitions=46,,,
,object,Int64,object
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [202]:
ddf_sgg_multi["cnt"] = 1
df_sgg_multi = (
    ddf_sgg_multi.groupby(["시도_코드", "시군구_코드"])
    .agg({"cnt": "sum", "지하_주택_전유부_수": "sum"})
    .compute()
)
df_sgg_multi["cat"] = "다세대_연립"
df_sgg_multi = df_sgg_multi.sort_values("시군구_코드")
df_sgg_multi


cnt  지하_주택_전유부_수     cat
시도_코드 시군구_코드                           
11    11110   2097         2320  다세대_연립
      11140    836         1799  다세대_연립
      11170   2240         2173  다세대_연립
      11200    947         1180  다세대_연립
      11215   1646         1695  다세대_연립
...            ...          ...     ...
48    48870      8            5  다세대_연립
      48880      9            8  다세대_연립
      48890      3            2  다세대_연립
50    50110   2191           80  다세대_연립
      50130    331           63  다세대_연립

[249 rows x 3 columns]

In [203]:
df_sido_multi = df_sgg_multi.groupby("시도_코드").agg(
    {"cnt": "sum", "지하_주택_전유부_수": "sum", "cat": "first"}
)
df_sido_multi["시군구_코드"] = df_sido_multi.index.str[:] + "000"
df_sido_multi


,cnt,지하_주택_전유부_수,cat,시군구_코드
시도_코드,,,,
11,51242,61585,다세대_연립,11000
26,6342,1033,다세대_연립,26000
27,2644,901,다세대_연립,27000
28,15250,20661,다세대_연립,28000
29,598,428,다세대_연립,29000
30,3138,611,다세대_연립,30000
31,1099,484,다세대_연립,31000
36,35,13,다세대_연립,36000
41,34277,21761,다세대_연립,41000


In [204]:
df_sido = pd.concat([df_sido_shared, df_sido_multi])
df_sido_pivot = df_sido.pivot(index=None, columns="cat", values=["cnt", "지하_주택_전유부_수"])
df_sido_pivot = df_sido_pivot.join(s_sido)
df_sido_pivot = df_sido_pivot[
    [
        "시도명",
        ("cnt", "다가구_다중"),
        ("cnt", "다세대_연립"),
        ("지하_주택_전유부_수", "다세대_연립"),
    ]
]

df_sido_pivot = df_sido_pivot.sort_index()
df_sido_pivot


c:\Users\USER\Documents\openeais\venv\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,시도명,"(cnt, 다가구_다중)","(cnt, 다세대_연립)","(지하_주택_전유부_수, 다세대_연립)"
시도_코드,,,,
11,서울특별시,107344,51242,61585
26,부산광역시,5337,6342,1033
27,대구광역시,7072,2644,901
28,인천광역시,11678,15250,20661
29,광주광역시,5462,598,428
30,대전광역시,12212,3138,611
31,울산광역시,842,1099,484
36,세종특별자치시,100,35,13
41,경기도,68727,34277,21761


In [205]:
df_sgg = pd.concat([df_sgg_shared, df_sgg_multi])
df_sgg = df_sgg.reset_index(0)

df_sgg_pivot = df_sgg.pivot(index=None, columns="cat", values=["cnt", "지하_주택_전유부_수"])
df_sgg_pivot = df_sgg_pivot.join(s_sgg)
df_sgg_pivot["시도_코드"] = df_sgg_pivot.index.str[:2]
df_sgg_pivot = df_sgg_pivot.join(s_sido, on="시도_코드")

df_sgg_pivot = df_sgg_pivot[
    [
        "시군구명",
        "시도_코드",
        "시도명",
        ("cnt", "다가구_다중"),
        ("cnt", "다세대_연립"),
        ("지하_주택_전유부_수", "다세대_연립"),
    ]
]

df_sgg_pivot = df_sgg_pivot.sort_index()
df_sgg_pivot


c:\Users\USER\Documents\openeais\venv\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,시군구명,시도_코드,시도명,"(cnt, 다가구_다중)","(cnt, 다세대_연립)","(지하_주택_전유부_수, 다세대_연립)"
시군구_코드,,,,,,
11110,서울특별시 종로구,11,서울특별시,1939,2097,2320
11140,서울특별시 중구,11,서울특별시,1172,836,1799
11170,서울특별시 용산구,11,서울특별시,3259,2240,2173
11200,서울특별시 성동구,11,서울특별시,2952,947,1180
11215,서울특별시 광진구,11,서울특별시,5397,1646,1695
...,...,...,...,...,...,...
48870,경상남도 함양군,48,경상남도,11,8,5
48880,경상남도 거창군,48,경상남도,7,9,8
48890,경상남도 합천군,48,경상남도,1,3,2


In [206]:
df_sido_pivot.to_csv("output/uground_sido.csv", encoding="utf_8_sig")
df_sgg_pivot.to_csv("output/uground_sgg.csv", encoding="utf_8_sig")


## 시도별 목록 추출

In [207]:
ddf_uground_shared["허가_연"] = ddf_uground_shared.허가_일.str[:4]
ddf_uground_shared["허가_월"] = ddf_uground_shared.허가_일.str[4:6]
ddf_uground_shared["허가_일자"] = ddf_uground_shared.허가_일.str[6:8]

ddf_uground_multi["허가_연"] = ddf_uground_multi.허가_일.str[:4]
ddf_uground_multi["허가_월"] = ddf_uground_multi.허가_일.str[4:6]
ddf_uground_multi["허가_일자"] = ddf_uground_multi.허가_일.str[6:8]


In [208]:
for code, name in s_sido.to_dict().items():
    print(code, name)


11 서울특별시
26 부산광역시
27 대구광역시
28 인천광역시
29 광주광역시
30 대전광역시
31 울산광역시
41 경기도
42 강원도
43 충청북도
44 충청남도
45 전라북도
46 전라남도
47 경상북도
48 경상남도
50 제주특별자치도
36 세종특별자치시


In [209]:
for code, name in s_sido.to_dict().items():
    ddf_uground_shared[ddf_uground_shared.시군구_코드.fillna("").str.startswith(code)].to_csv(
        f"output/uground_sido/{name}_shared.csv", single_file=True, encoding="utf_8_sig"
    )
    ddf_uground_multi[ddf_uground_multi.시군구_코드.fillna("").str.startswith(code)].to_csv(
        f"output/uground_sido/{name}_multi.csv", single_file=True, encoding="utf_8_sig"
    )
